### 1. Binary Cross Entropy Loss (BCE)

- ใช้สำหรับงาน Binary Classification หรือการจำแนกที่มีเพียง 2 ตัวเลือก (เช่น เป็นสแปม หรือ ไม่เป็นสแปม) 
Activation Function: มักใช้คู่กับ Sigmoid ในเลเยอร์สุดท้าย ซึ่งจะให้ค่าความน่าจะเป็นระหว่าง 0 ถึง 1 สำหรับคลาสบวก (Positive Class)

![alt text](<สกรีนช็อต 2025-12-26 015526.png>)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('churn_data.csv')
df = pd.get_dummies(df, columns=['ContractType'], drop_first=True)
X = df.drop('Churn', axis=1).values
y = df['Churn'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

In [3]:
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

import torch.nn as nn

class ChurnNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

model = ChurnNet(input_dim=X_train.shape[1])


In [4]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = torch.sigmoid(model(inputs))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.7652
Epoch 2, Loss: 0.7617
Epoch 3, Loss: 0.7582
Epoch 4, Loss: 0.7547
Epoch 5, Loss: 0.7513
Epoch 6, Loss: 0.7478
Epoch 7, Loss: 0.7444
Epoch 8, Loss: 0.7410
Epoch 9, Loss: 0.7376
Epoch 10, Loss: 0.7343
